<a href="https://colab.research.google.com/github/cllamas107/imersaogemini2025/blob/main/ArenaUPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [3]:
# Configura a API Key
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [25]:
# Configura o cliente da SDK

from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"


In [18]:
!pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [19]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [20]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [21]:
##########################################################
# --- Agente 1: Crítico busca resposta do Consumidor --- #
##########################################################
def agente_critico(topico, data_de_hoje):
    critico = Agent(
        name="agente_critico",
        model="gemini-2.0-flash",
        description="Agente Critico que busca respostas no google",
        tools=[google_search],
        instruction="""
        Você é um especialista de perfil acadêmico, engenheiro de alimentos, pesquisador. A sua tarefa é procurar a resposta para a pergunta
        (google_search), analisar e preparar a sua melhor resposta para o usuário.
        Sua resposta deve ser em português e será analisada pelo agente juiz que definira qual a melhor resposta, se a do crítico
        ou do agente contraponto. Apresentar riscos e benefícios (ou nuances), permitirão explorar a complexidade do tema dos
        alimentos ultraprocessados de maneira equilibrada.
        Você é defensor de que os alimentos ultraprocessados prejudicam a saúde e causam doenças.
        """
    )
    entrada_do_agente_critico = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    resposta_critico = call_agent(critico, entrada_do_agente_critico)
    return resposta_critico

In [28]:
#############################################################
# --- Agente 2: Contraponto busca resposta do Consumidor--- #
#############################################################
def agente_contraponto(topico, data_de_hoje):
    contraponto = Agent(
        name="agente_contraponto",
        model="gemini-2.0-flash",
        description="Agente Contraponto que busca respostas no google",
        tools=[google_search],
        instruction="""
        Você é um especialista de perfil acadêmico, engenheiro de alimentos, pesquisador. A sua tarefa é procurar a resposta para a pergunta
        (google_search), analisar e preparar a sua melhor resposta para o usuário consumidor.
        Sua resposta deve ser em português e será analisada pelo agente juiz que definira qual a melhor resposta, se a do crítico
        ou do agente contraponto.
        Você é crítico a vilanização de alimentos industrializados e tenta argumentar que nem todos os ataques feitos aos alimentos ultraprocessados
        tem base científica sólida, e que existem muita campanha difamatória
        de organizações não governamentais atribuindo exageradamente muitas doenças aos UPFs
        """,
    )

    entrada_do_agente_contraponto = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    resposta_do_contraponto = call_agent(contraponto, entrada_do_agente_contraponto)
    return resposta_do_contraponto

In [26]:
######################################
# --- Agente 3: Juiz --- #
######################################
def agente_juiz(topico, resposta_critico, resposta_contraponto):
    juiz = Agent(
        name="agente_juiz",
        model="gemini-2.0-flash",
        instruction="""
            Você será o juiz que avaliara as duas respostas do agente critico e contraponto.
            O critério primordial deve ser a qualidade e a relevância das evidências científicas apresentadas.
            O Agente Juiz precisará identificar se as
            alegações são suportadas por estudos revisados por pares, metanálises,
            diretrizes de organizações de saúde renomadas, etc.
            Analise as duas respostas abaixo com base nos seguintes critérios:
            - Quão bem a resposta é suportada por evidências científicas?
            - Quão clara e coerente é a argumentação?
            - Quão relevante é a resposta para a pergunta do Consumidor?
            - Quão forte e confiável são as evidências apresentadas?
            Atribua um ponto para a resposta que melhor atender a esses critérios e explique seu raciocínio.
            responda da seguinte forma:
            O especialista (critico ou contraponto) apresentou a melhor resposta.
            """,
        description="Juiz especialista acadêmico que analisa as respostas dos agentes critico e contraponto",
    )
    entrada_do_agente_juiz = f"Tópico: {topico}\nCrítico: {resposta_critico}\n {resposta_contraponto}"
    # Executa o agente
    veredito = call_agent(juiz, entrada_do_agente_juiz)
    return veredito

In [29]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🥊 Arena UPF 🥊\n")
print("Você se sente confuso com tantas notícias sobre alimentos ultraprocessados? \nCuidar da saúde é fundamental, e entender o que comemos faz toda a diferença.\n ")

# --- Obter o Tópico do Usuário ---
topico = input("❓Faça sua pergunta e veja o confronto de ideias entre nossos agentes especialistas! ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Preciso da sua pergunta para continuar!")
else:
  print("Maravilha! Agora vamos deixar os entendidos trabalharem!")

  resposta_critico = agente_critico(topico, data_de_hoje)
  print("\n🎯 Resposta Crítico! 🎯\n")
  display(to_markdown(resposta_critico))
  print("--------------------------------------------------")

  resposta_contraponto = agente_contraponto(topico, data_de_hoje)
  print("\n✍️ Resposta Contraponto ✍️\n")
  display(to_markdown(resposta_contraponto))
  print("--------------------------------------------------")

  veredito = agente_juiz(topico, resposta_critico, resposta_contraponto)
  print("\n🚀 Resultado Juiz 🚀\n")
  display(to_markdown(veredito))
  print("--------------------------------------------------")



🥊 Arena UPF 🥊

Você se sente confuso com tantas notícias sobre alimentos ultraprocessados? 
Cuidar da saúde é fundamental, e entender o que comemos faz toda a diferença.
 
❓Faça sua pergunta e veja o confronto de ideias entre nossos agentes especialistas! Por que alimentos ultraprocessados ainda são vendidos livremente se dizem que fazem mal à saúde?
Maravilha! Agora vamos deixar os entendidos trabalharem!

🎯 Resposta Crítico! 🎯



> Aqui estão algumas das razões pelas quais os alimentos ultraprocessados ainda são vendidos livremente, apesar de serem considerados prejudiciais à saúde:
> 
> 
> Embora haja um crescente consenso sobre os malefícios dos alimentos ultraprocessados para a saúde, eles ainda são amplamente vendidos por diversas razões complexas e interconectadas:
> 
> *   **Disponibilidade e Acessibilidade:** Em muitas comunidades, especialmente em áreas de baixa renda e desertos alimentares, os alimentos ultraprocessados são mais acessíveis e baratos do que opções frescas e saudáveis. Isso ocorre devido a fatores como subsídios governamentais à agricultura industrial e a logística de distribuição em larga escala.
> 
> *   **Marketing e Publicidade:** A indústria de alimentos ultraprocessados investe fortemente em marketing e publicidade, criando desejos e hábitos de consumo desde a infância. As táticas incluem embalagens atraentes, promoções de preços e direcionamento específico para grupos vulneráveis.
> 
> *   **Prazo de Validade e Conveniência:** Alimentos ultraprocessados têm um longo prazo de validade e são convenientes para pessoas com pouco tempo para cozinhar. Isso os torna uma opção atraente para muitos consumidores, apesar dos riscos à saúde.
> 
> *   **Falta de Regulamentação Eficaz:** A regulamentação sobre a produção, comercialização e rotulagem de alimentos ultraprocessados ainda é insuficiente em muitos países. Isso permite que a indústria continue a promover produtos nocivos sem restrições significativas.
> 
> *   **Interesses Econômicos:** A indústria de alimentos ultraprocessados é uma força econômica poderosa, com grande influência sobre governos e legisladores. Isso dificulta a implementação de políticas públicas que visem restringir a produção e o consumo desses alimentos.
> 
> *   **Falta de Conscientização:** Apesar do crescente corpo de evidências científicas, muitos consumidores ainda não estão plenamente conscientes dos riscos associados ao consumo de alimentos ultraprocessados. A desinformação e a falta de educação nutricional contribuem para esse problema.
> 
> É crucial que os governos implementem políticas públicas eficazes, como regulamentação mais rigorosa, impostos sobre alimentos não saudáveis, subsídios para alimentos frescos e campanhas de educação nutricional, para proteger a saúde da população e reduzir o consumo de alimentos ultraprocessados.


--------------------------------------------------

✍️ Resposta Contraponto ✍️



> Compreendo sua pergunta sobre por que alimentos ultraprocessados (UPFs) continuam amplamente disponíveis no mercado, mesmo com as preocupações sobre seus potenciais impactos negativos à saúde. A questão é complexa e envolve diversos fatores, desde econômicos e sociais até científicos e regulatórios.
> 
> Para te dar uma resposta mais completa, vou pesquisar no Google sobre os seguintes pontos:
> 
> 
> Após analisar as informações disponíveis, apresento uma visão geral sobre a questão dos alimentos ultraprocessados e sua venda livre:
> 
> **1. Razões para a ampla disponibilidade:**
> 
> *   **Acessibilidade e preço:** UPFs são frequentemente mais baratos e acessíveis do que alimentos frescos ou minimamente processados, especialmente para populações de baixa renda.
> *   **Conveniência:** São produtos fáceis de consumir, exigindo pouco ou nenhum preparo, o que atrai consumidores com pouco tempo disponível.
> *   **Marketing agressivo:** A indústria de alimentos ultraprocessados investe pesadamente em marketing e publicidade, criando desejo e normalizando o consumo desses produtos, incluindo estratégias direcionadas a crianças em idade escolar (Referências 5, 8, 18, 19).
> *   **Impacto econômico:** A indústria alimentícia é um setor econômico poderoso, com grande influência política e econômica, o que dificulta a implementação de regulamentações mais restritivas (Referências 4, 5).
> *   **Globalização e políticas de mercado:** A globalização facilitou a expansão das grandes corporações transnacionais, que dominam a produção e distribuição de UPFs, enquanto políticas de desregulamentação e abertura ao investimento estrangeiro impulsionaram o crescimento desse mercado (Referências 5, 9, 21).
> 
> **2. Impactos negativos à saúde (com nuances):**
> 
> *   **Evidências crescentes:** Há um corpo crescente de evidências que associa o consumo elevado de UPFs ao aumento do risco de obesidade, diabetes tipo 2, doenças cardiovasculares, câncer e outros problemas de saúde (Referências 2, 7, 9, 14, 15, 23, 24, 25).
> *   **Dificuldade em comprovar causalidade:** Apesar das associações, provar que os UPFs são a causa *direta* desses problemas é um desafio científico complexo (Referências 7, 10). Estudos observacionais podem ser influenciados por outros fatores de estilo de vida e dificuldades em isolar os efeitos específicos dos UPFs.
> *   **Qualidade nutricional:** UPFs são geralmente ricos em gorduras saturadas, açúcar, sódio e aditivos químicos, enquanto são pobres em fibras, vitaminas e minerais (Referências 3, 14, 20). Essa composição nutricional desequilibrada contribui para o desenvolvimento de doenças crônicas.
> *   **Substituição de alimentos saudáveis:** O consumo de UPFs frequentemente leva à substituição de alimentos *in natura* ou minimamente processados, que são fontes importantes de nutrientes essenciais (Referências 9, 24).
> 
> **3. O que diz a lei e a regulamentação:**
> 
> *   **Rotulagem nutricional:** No Brasil, a Agência Nacional de Vigilância Sanitária (ANVISA) exige que alimentos embalados apresentem informações nutricionais detalhadas nos rótulos, incluindo a lista de ingredientes e valores diários recomendados (Referências 2, 3, 16).
> *   **Nova rotulagem frontal:** Desde 2022, está em vigor a nova rotulagem nutricional frontal, que exige a inclusão de um símbolo de lupa na parte frontal da embalagem para alertar os consumidores sobre o alto teor de açúcar, sódio ou gordura saturada adicionada (Referências 3, 6, 14).
> *   **Restrições em ambientes escolares:** Há restrições à compra de UPFs com recursos do Programa Nacional de Alimentação Escolar (PNAE), que proíbe a inclusão de bebidas açucaradas e outros produtos ultraprocessados no cardápio da rede pública de ensino (Referência 6).
> *   **Regulamentação em outros países:** Alguns países, como o Chile, implementaram sistemas de rotulagem frontal mais rigorosos e restrições à publicidade de UPFs direcionada a crianças (Referência 4).
> *   **Debates e iniciativas:** Há debates e iniciativas em andamento para incluir advertências nos rótulos de UPFs com altos teores de açúcar, sódio e gorduras saturadas, além de medidas restritivas sobre a publicidade desses produtos (Referências 2, 6, 14).
> 
> **4. O papel da indústria e o lobby:**
> 
> *   **Influência política:** A indústria de alimentos ultraprocessados exerce grande influência nas decisões políticas, utilizando recursos econômicos e humanos para impedir a implementação de regulamentações mais restritivas (Referências 4, 18).
> *   **Estratégias de marketing:** As empresas de UPFs utilizam diversas estratégias de marketing para promover seus produtos, incluindo publicidade direcionada a crianças, programas de responsabilidade social corporativa e ações de "lavagem de imagem" (Referências 5, 8, 18, 19).
> *   **Contestação de evidências científicas:** A indústria frequentemente contesta as evidências científicas que relacionam o consumo de UPFs a problemas de saúde, argumentando que a classificação é imprecisa e que o consumo desses alimentos é apenas um dos muitos fatores a serem ponderados (Referências 15, 22).
> 
> **5. O que pode ser feito?**
> 
> *   **Políticas públicas:** É necessário implementar políticas públicas que promovam o acesso a alimentos saudáveis e restrinjam a produção, distribuição e comercialização de UPFs, como a tributação de UPFs e a regulamentação da publicidade (Referências 4, 6, 13, 14, 17).
> *   **Educação alimentar:** É fundamental investir em educação alimentar para informar os consumidores sobre os riscos do consumo de UPFs e promover escolhas alimentares mais conscientes (Referência 1).
> *   **Fortalecimento da agricultura familiar:** Apoiar a agricultura familiar e os sistemas alimentares locais pode aumentar a disponibilidade e o acesso a alimentos frescos e saudáveis (Referência 6).
> *   **Reformulação de produtos:** Incentivar a indústria a reformular seus produtos, reduzindo o teor de açúcar, sódio, gorduras saturadas e aditivos químicos (Referência 1).
> *   **Ações multidisciplinares:** Promover ações contínuas e multidisciplinares na comunidade para promover hábitos alimentares mais saudáveis, considerando as características socioeconômicas e culturais da população (Referência 24).
> 
> **Em resumo:** A venda livre de alimentos ultraprocessados é resultado de uma combinação complexa de fatores econômicos, sociais, políticos e culturais. Embora haja evidências crescentes dos seus impactos negativos à saúde, a indústria exerce grande influência e resiste à implementação de regulamentações mais restritivas. Para reverter essa situação, é necessário um esforço conjunto de governos, sociedade civil, profissionais de saúde e da própria indústria para promover sistemas alimentares mais saudáveis e sustentáveis.
> 
> É importante ressaltar que nem todos os alimentos industrializados são necessariamente prejudiciais à saúde. O problema reside no consumo excessivo de *ultraprocessados*, que são produtos com alta densidade calórica, pobres em nutrientes e ricos em aditivos químicos.


--------------------------------------------------

🚀 Resultado Juiz 🚀



> O especialista contraponto somou 1 ponto.
> 
> Racional:O especialista contraponto apresentou uma resposta mais completa, abordando todos os aspectos levantados na pergunta do consumidor de forma clara e organizada. Além disso, ele forneceu evidências científicas e dados relevantes para embasar seus argumentos, tornando sua resposta mais confiável e informativa. A resposta do especialista crítico, embora relevante, não se aprofundou tanto nos detalhes e não apresentou um embasamento científico tão robusto quanto o do especialista contraponto.
> 


--------------------------------------------------
